# PGFinder Interactive Notebook

This notebook is a basic user interface to allow researchers less familiar with the command line to run PGFinder. Some compromises have been made to make it run as a Jupyter notebook on the free myBinder platform.

To use the code to analyse your data you must work from top to bottom on this notebook, following instructions as you go. The first step is to execute all of the "cells" in the notebook so they are ready for your input. To do this...

**Click *Kernel*>*Restart & Run All* on the menu, above.**

If you have any problems or suggestions, or would like to contribute a mass database, please raise an [issue here](https://github.com/Mesnage-Org/pgfinder/issues). This notebook runs on the latest release. Please review the [release notes](https://github.com/Mesnage-Org/pgfinder/releases).

In [1]:
import base64
import codecs
import io
import os
import uuid
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display
from ipysheet import from_dataframe
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pgfinder import matching, pgio, validation

# Get list of modifications
allowed_mods = validation.allowed_modifications()

# Get built in mass lists
mass_lists_path = './data/masses'
mass_lists = os.listdir(mass_lists_path)
mass_lists.append('Upload Custom')

# Main analysis function
def analysis(b):
    # Upload deconvoluted file
    # This widget returns a dictionary of details of uploaded files
    # However, the widget is restricted to allow only one file
    # Hence `.value[0]`
    uploaded_df = pgio.ms_upload_reader(data_uploader.children[0].value[0])
    
    # Load mass list
    if rb_masses.value == 'Upload Custom':
        theo_masses = pgio.theo_masses_upload_reader(mass_uploader.children[0].value[0])
    else:
        csv_filepath = os.path.join(mass_lists_path, rb_masses.value)
        theo_masses = pgio.theo_masses_reader(csv_filepath)

    # Load ppm value
    user_ppm = ppm_tol.value

    # Load time delta value
    user_time_delta = time_delta.value

    # Load decimal places
    dp = decimal_places.value
    # Make sure mod list is a list
    mod_list = list(selector_mods.value)
    
    results = matching.data_analysis(uploaded_df, theo_masses, user_time_delta, mod_list, user_ppm)
    consolidated = matching.consolidate_matches(results)
          
    # Make the download button(s)
    def _download_results(df: pd.DataFrame, filename_prefix: str, float_format: int=4):
        """Create a download button for the provided DataFrame as CSV.
        
        Parameters
        ----------
        df: pd.DataFrame
            Pandas DataFrame to be downloaded.
        filename: str
            Filename to prefix output with.
        float_format: int
            Formatting for floating point numbers, default is 4 decimal places.
        """
        filename = pgio.default_filename(prefix=filename_prefix)
        if filename_prefix == "consolidated_":
            results_csv_str = df.to_csv(float_format="%d")
            download_file = "Download Consolidated"
        else:
            results_csv_str = pgio.dataframe_to_csv_metadata(output_dataframe=df, float_format=f"%.{str(float_format)}f")
            download_file = "Download Results File"
        b64 = base64.b64encode(results_csv_str.encode())
        payload = b64.decode()
        html_buttons = '''<html>
            <head>
            <meta name="viewport" content="width=device-width, initial-scale=1">
            </head>
            <body>
            <a download="{filename}" href="data:text/csv;base64,{payload}" download>
            <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">{download_file}</button>
            </a>
            </body>
            </html>
            '''

        html_button = html_buttons.format(payload=payload,filename=filename, download_file=download_file)
        return display(HTML(html_button))

    _download_results(df=results, filename_prefix="results_", float_format=dp)
    _download_results(df=consolidated, filename_prefix="consolidated_")

    
# Define widgets

# A composite widget for picking a file and displaying its name
def named_file_upload(accept, description):
    file_upload = widgets.FileUpload(
        accept = accept, 
        description = description,
        multiple = False,
        layout = big_button
    )
    file_name = widgets.Label(value="No file selected...")
    def handle_file_upload(file):
        file_name.value = file["new"][0]["name"]
    file_upload.observe(handle_file_upload, names='value')
    return widgets.HBox([file_upload, file_name])

# Layout for a bigger button
big_button = widgets.Layout(width='auto')

# Style for wider description
wide_style = {'description_width': 'initial'}

# Deconvoluted Data file upload
data_uploader = named_file_upload('.txt,.ftrs', 'Upload Deconvoluted Data')

# Modifcation selector
selector_mods = widgets.SelectMultiple(
    options = allowed_mods,
    description = 'Modification',
    disabled = False
)

# Mass library selector
rb_masses = widgets.RadioButtons(
    options = mass_lists,
    description = 'Mass List',
    disabled = False
)

# Mass library file upload
mass_uploader = named_file_upload('.csv', 'Upload Mass Library')

# Set PPM tolerance
ppm_tol = widgets.BoundedFloatText(
    value = 10.0,
    min = 1,
    max = 100,
    step = 0.1,
    description = 'Set ppm tolerance',
    disabled = False,
    style = wide_style
    )

# Set time delta for in source clean up
time_delta = widgets.BoundedFloatText(
    value = 0.5,
    min = 0,
    max = 100,
    step = 0.01,
    description = 'Set time delta value',
    disabled = False,
    style = wide_style
)

# Decimal places
decimal_places = widgets.Dropdown(
    options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    value=4,
    description='Decimal Places:',
    disabled=False,
)

# Analysis button
button = widgets.Button(description="Run Analysis")
button.on_click(analysis)

/tmp/ipykernel_202224/2655535379.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


[Fri, 05 May 2023 21:34:34] [INFO    ] [pgfinder] Loaded parameters from file : config/parameters.yaml
[Fri, 05 May 2023 21:34:34] [INFO    ] [pgfinder] All parameters converted to decimal


## Step 1: Upload Deconvoluted Data
Click *Upload* to upload a `.txt` file output by MaxQuant ([example file](https://github.com/Mesnage-Org/pgfinder/raw/master/data/maxquant_test_data.txt)), or an `.ftrs` file.

In [2]:
display(data_uploader)

## Step 2: Select Modifications
Select modifications (Hold down control / command and click to select mulitple items.)

In [3]:
display(selector_mods) 

SelectMultiple(description='Modification', options=('Sodium', 'Potassium', 'Anh', 'DeAc', 'DeAc_Anh', 'Nude', …

## Step 3: Select or Upload Mass Library

### Select

In [4]:
display(rb_masses)

RadioButtons(description='Mass List', options=('e_coli_monomer_masses.csv', 'c_diff_monomer_masses.csv', 'Uplo…

### (Optional) Upload Custom Mass Library
[Example mass library file.](https://raw.githubusercontent.com/Mesnage-Org/pgfinder/master/data/masses/e_coli_monomer_masses.csv)

In [5]:
display(mass_uploader)

## Step 4: Set PPM tolerance

In [6]:
display(ppm_tol)

BoundedFloatText(value=10.0, description='Set ppm tolerance', min=1.0, step=0.1, style=DescriptionStyle(descri…

## Step 5: Set time window for in-source decay and salt adduct clean up

In [7]:
display(time_delta)

BoundedFloatText(value=0.5, description='Set time delta value', step=0.01, style=DescriptionStyle(description_…

## Step 6: Set decimal places for Output

If you want a different number of decimal places in the full results output than 4 you can change this below.

In [8]:
display(decimal_places)

Dropdown(description='Decimal Places:', index=3, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=4)

## Step 7: Run Analysis
Click run analysis.
After the analysis is complete, a download button will appear.

In [9]:
display(button)

Button(description='Run Analysis', style=ButtonStyle())

[Fri, 05 May 2023 21:35:01] [INFO    ] [pgfinder] Mass spectroscopy file loaded from  : WT.ftrs
   ID  Ion count Charge state  XIC start (min)  XIC end (min)  RT (min)   
0   1         67            1         0.007527       1.175287  0.614847  \
1   2        134            1         0.007527       1.190458  0.471928   
2   3         97            1         0.007527       1.175287  0.647689   
3   4         54            1         0.007527       0.818386  0.280048   
4   5         82            1         0.007527       1.175287  0.565021   

     Obs (Da)  Theo (Da)  Inferred structure  Intensity  
0  183.068895        NaN                 NaN    1188358  
1  199.964876        NaN                 NaN     766035  
2  227.028734        NaN                 NaN     430046  
3  153.959490        NaN                 NaN     183453  
4  254.224276        NaN                 NaN      49490  
        ID  Ion count Charge state  XIC start (min)  XIC end (min)   RT (min)   
7759  7760          4   

~~~~~~~~~~ obs_monomers_df ~~~~~~~~~~
STATEMENT : [data_analysis] : BEFORE CONCAT
Shape           :
(60, 2)
Head            :
     Inferred structure  Theo (Da)
2815          GM (x4)|0  1932.7458
918                GM|0   498.2061
1996            GM-AE|1   698.2858
1475           GM-AEJ|1   870.3706
2000          GM-AEJA|1   941.4077
Tail            :
     Inferred structure  Theo (Da)
2113         GM-AEJVS|1  1056.4753
5143          GM-AEJW|1  1056.4553
2050         GM-AEJWW|1  1242.5346
3413          GM-AEJY|1  1033.4353
3180         GM-AEJYG|1  1090.4568
~~~~~~~~~~ obs_multimers_df ~~~~~~~~~~
STATEMENT : [data_analysis] : BEFORE CONCAT
Shape           :
(0, 0)
Head            :
Empty DataFrame
Columns: []
Index: []
Tail            :
Empty DataFrame
Columns: []
Index: []
~~~~~~~~~~ obs_theo_df ~~~~~~~~~~
STATEMENT : [data_analysis] : AFTER CONCAT should be sum of previous two
Shape           :
(60, 2)
Head            :
  Inferred structure  Theo (Da)
0          GM (x4)|0  1932.7458
1